In [1]:
import numpy as np
import tensorflow as tf
from utils_functions import *
from plan import *
import random
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
import datetime, os




In [2]:

dict_stati= load_file("./dizionario_stati")
plansLoaded=load_file("./plans")
#print(type(plansPickles))
#plansList=load_from_pickles("C:/Users/Francesco/Desktop/dataset/dataset/dizionario_stati",plansPickles)
#print(plansPickles)


File loaded
File loaded


In [3]:
#ogni stato è un tensore con elementi dtype int8, questi stati vengono raggruppati in un altro tensore che rappresnta la
#variabile "states" del piano 
def build_vector(dict, states_list):
    l=len(dict)
    vector_states=[]
    for state in states_list:
        vector=np.array([0]*l,dtype=np.int8)
        #vector=tf.zeros(l,dtype=np.int8)
        for s in state:
            for key in dict.keys():
                if key==s:
                    vector[dict[key]-1]=1
                    break
        #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
        vector_states.append(vector) 
    r = np.array(vector_states)                          
    return r


#NON UTILIZZATO ADESSO
#shape di ogni singolo elemento (r) è (n x 340) con n che varia su ogni piano, raggruppati sulla base dei piani
def build_all_vectors(dict,plans_list):
    total=[]
    for plan in plans_list:
        r=build_vector(dict,plan.states)
        total.append(r)
    #r = tf.convert_to_tensor(total,dtype=None)                         
    return total
 
    
#gli stati vengono ordinati (in una lista) singolarmente con shape (1x340), non vengono raggruppati sulla base dei piani
#da utilizzare per autoencoder standard
def build_all_vectors1x340(dict, plans_list):
    l=len(dict)
    total=[]
    for plan in plans_list:
        for state in plan.states:
            vector=np.array([0]*l,dtype=np.int8)
            for s in state:
                for key in dict.keys():
                    if key==s:
                        vector[dict[key]-1]=1
                        break
            #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
            total.append(vector)
    total=np.array(total)        
    return total        

In [4]:
#all_states_from_plans_list=build_all_vectors(dict_stati,plansLoaded)

## Preparazione dati

In [5]:
#Creazione dataset totale
all1x340=build_all_vectors1x340(dict_stati,plansLoaded)
random.shuffle(all1x340)


In [6]:
print(all1x340.shape)
print(type(all1x340))
print(type(all1x340[6]))
print(all1x340[6].dtype)

(1384373, 340)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
int8


In [7]:
#creAzione di train, test e validation set,--> input shape per rete = 340
#rendere questi set statici, altimenti cambiano sempre ad ogni esecuzione.... es. comprimerli con 7zip o pickle --> DA FARE IN PRIMA POSSIBILE
#random.shuffle(all1x340)
half=int(len(all1x340)//3)
train=all1x340[half:]
vt=all1x340[:half]
h2=int(len(vt)//2)
validation=vt[h2:]
test=vt[:h2]


In [8]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

## Modello

In [9]:
input_size = 340 #costante

#Scelti casulmente al momento.... da rivedere come prima modifica forse
hidden_size = 85
#hidden_size2 = 85
code_size = 17



my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=20),
    tf.keras.callbacks.TensorBoard(log_dir="C:/Users/lucal/PycharmProjects/Deep_Learning_Autoencoder/logsTB",histogram_freq=1)

]

#Cose da fare per migliorare la rete solo dopo aver fatto i primi tentativi con la rete proposta:
# * Provare ad aggiungere Regolarizzazione es. L1,L2 e dropout(solo nella fase di encoding)
# * Provare swish al posto di relu
# * Provare keras.layers.BatchNormalization()
# * Fare Hyperparameter Tuning  (ultima) www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams
#Salvare i risultati con ogni modifica fatta per scriverli nel report

input_layer = Input(shape=(input_size,))
hidden_1 = Dense(hidden_size, activation='relu', kernel_initializer="he_uniform")(input_layer)
#hidden_3 = Dense(hidden_size, activation='relu', kernel_initializer="he_uniform")(hidden_1)

code = Dense(code_size, activation='relu',kernel_initializer="he_uniform")(hidden_1)

hidden_2 =Dense(hidden_size, activation='relu',kernel_initializer="he_uniform")(code)
#hidden_4 =Dense(hidden_size, activation='relu',kernel_initializer="he_uniform")(hidden_2)
output_layer = Dense(input_size, activation='sigmoid')(hidden_2)

autoencoder = Model(input_layer, output_layer)
#variare learning_rate beta_1,beta_2, da fare per ultimo
#batch_size ???
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='binary_crossentropy',metrics=["accuracy","Precision","Recall"])
autoencoder.fit(x=train,y=train, epochs=5000,batch_size=5000, validation_data=(validation,validation), callbacks=my_callbacks,
                      )

 #Gardare i risulati delle metriche su tensorbord                      

Epoch 1/5000
185/185 [==============================] - 4s 11ms/step - loss: 0.1762 - accuracy: 0.0565 - precision: 0.3992 - recall: 0.0478 - val_loss: 0.1641 - val_accuracy: 0.0932 - val_precision: 0.7510 - val_recall: 0.0717
Epoch 2/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1630 - accuracy: 0.0595 - precision: 0.7318 - recall: 0.0787 - val_loss: 0.1627 - val_accuracy: 0.0761 - val_precision: 0.7226 - val_recall: 0.0793
Epoch 3/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1624 - accuracy: 0.0635 - precision: 0.7189 - recall: 0.0833 - val_loss: 0.1624 - val_accuracy: 0.0617 - val_precision: 0.7049 - val_recall: 0.0921
Epoch 4/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1620 - accuracy: 0.0553 - precision: 0.6972 - recall: 0.0961 - val_loss: 0.1620 - val_accuracy: 0.0329 - val_precision: 0.6958 - val_recall: 0.0963
Epoch 5/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1617 - accuracy: 

185/185 [==============================] - 1s 7ms/step - loss: 0.1581 - accuracy: 0.0249 - precision: 0.6943 - recall: 0.0874 - val_loss: 0.1582 - val_accuracy: 0.0222 - val_precision: 0.6960 - val_recall: 0.0866
Epoch 38/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1580 - accuracy: 0.0195 - precision: 0.7033 - recall: 0.0863 - val_loss: 0.1580 - val_accuracy: 0.0063 - val_precision: 0.7092 - val_recall: 0.0841
Epoch 39/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1580 - accuracy: 0.0226 - precision: 0.7058 - recall: 0.0859 - val_loss: 0.1581 - val_accuracy: 0.0219 - val_precision: 0.6977 - val_recall: 0.0870
Epoch 40/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1579 - accuracy: 0.0207 - precision: 0.7053 - recall: 0.0855 - val_loss: 0.1576 - val_accuracy: 0.0095 - val_precision: 0.6996 - val_recall: 0.0849
Epoch 41/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1574 - accuracy: 0.0187 - p

185/185 [==============================] - 1s 7ms/step - loss: 0.1571 - accuracy: 0.0278 - precision: 0.6909 - recall: 0.0931 - val_loss: 0.1572 - val_accuracy: 0.0883 - val_precision: 0.6900 - val_recall: 0.0932
Epoch 74/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1571 - accuracy: 0.0201 - precision: 0.6913 - recall: 0.0928 - val_loss: 0.1594 - val_accuracy: 0.0806 - val_precision: 0.6491 - val_recall: 0.0911
Epoch 75/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1572 - accuracy: 0.0192 - precision: 0.6917 - recall: 0.0924 - val_loss: 0.1577 - val_accuracy: 0.0055 - val_precision: 0.6950 - val_recall: 0.0870
Epoch 76/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1572 - accuracy: 0.0259 - precision: 0.6908 - recall: 0.0925 - val_loss: 0.1583 - val_accuracy: 0.0063 - val_precision: 0.7179 - val_recall: 0.0815
Epoch 77/5000
185/185 [==============================] - 1s 7ms/step - loss: 0.1573 - accuracy: 0.0282 - p

In [10]:
#%tensorboard --logdir logsTf
%load_ext tensorboard
#Avvio da terminale: spostarsi nella cartella dove è presente la cartella di log, > tensorboard --logdir nomecartella

# Print di test

In [11]:
print(validation[59])
print(test[2000])
print(train[5000])

[0 1 1 0 0 0 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 

In [12]:
for i in range(3):
    print(all_states_from_plans_list[i])

NameError: name 'all_states_from_plans_list' is not defined

In [ ]:
print("********initial state *******")
print(len(plansLoaded[1].initial_state))
print("")
print(plansLoaded[1].initial_state)

print("******** states *******")
print(len(plansLoaded[1].states))
print("")
print(plansLoaded[1].states)
print("******* goals ********")
print(len(plansLoaded[1].goals))
print("")
print(plansLoaded[1].goals)

print("\n\n -------------  dict stati -----------")
print(len(dict_stati))
print("")
print(dict_stati)

In [ ]:
for i in range(5):
    print(all1x340[i])